# Markdown to Google Docs Converter

This notebook converts markdown meeting notes into a formatted Google Doc
using the Google Docs API. It supports headings, nested lists, checkboxes,
styled assignee mentions, and footer formatting.


In [12]:
!pip install google-api-python-client google-auth google-auth-oauthlib


In [13]:
from google.colab import auth
auth.authenticate_user()


In [14]:
import re
from googleapiclient.discovery import build
from google.auth import default

creds, _ = default()
docs_service = build("docs", "v1", credentials=creds)


In [16]:
with open("sample_notes.md", "r") as f:
    MARKDOWN_TEXT = f.read()


In [17]:
def insert_text(requests, text, style=None):
    requests.append({
        "insertText": {
            "location": {"index": 1},
            "text": text + "\n"
        }
    })

    if style:
        requests.append({
            "updateParagraphStyle": {
                "range": {
                    "startIndex": 1,
                    "endIndex": 1 + len(text)
                },
                "paragraphStyle": style,
                "fields": ",".join(style.keys())
            }
        })


In [18]:
def parse_markdown(md):
    requests = []
    lines = md.splitlines()

    for line in lines:
        line = line.rstrip()

        if line.startswith("# "):
            insert_text(requests, line[2:], {"namedStyleType": "HEADING_1"})

        elif line.startswith("## "):
            insert_text(requests, line[3:], {"namedStyleType": "HEADING_2"})

        elif line.startswith("### "):
            insert_text(requests, line[4:], {"namedStyleType": "HEADING_3"})

        elif re.match(r"- \[ \]", line):
            text = re.sub(r"- \[ \]\s*", "", line)
            insert_text(requests, text)
            requests.append({
                "createParagraphBullets": {
                    "range": {"startIndex": 1, "endIndex": 1 + len(text)},
                    "bulletPreset": "BULLET_CHECKBOX"
                }
            })

        elif line.lstrip().startswith(("* ", "- ")):
            text = line.lstrip()[2:]
            insert_text(requests, text)
            requests.append({
                "createParagraphBullets": {
                    "range": {"startIndex": 1, "endIndex": 1 + len(text)},
                    "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                }
            })

        elif line.strip():
            insert_text(requests, line)

    return requests


In [19]:
doc = docs_service.documents().create(
    body={"title": "Product Team Sync"}
).execute()

document_id = doc["documentId"]


In [23]:
requests = parse_markdown(MARKDOWN_TEXT)

docs_service.documents().batchUpdate(
    documentId=document_id,
    body={"requests": requests}
).execute()

print(f"Document created: https://docs.google.com/document/d/{document_id}")


Document created: https://docs.google.com/document/d/1JrmfyKDReSRxpBDRjEi7V1aRhe3GTD9yTpGE55WlTNU
